In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings("ignore")
import featuretools as ft

import gc

In [140]:
train = pd.read_csv('../data/train_merged.csv')
test = pd.read_csv('../data/test_merged.csv')

In [141]:
train.shape, test.shape

((456548, 15), (32573, 14))

<h1>Manual Feature Engineering</h1>

In [142]:
#drop the outlier
train.drop(train[train.num_orders > 23000].index, inplace=True)

train['discount'] = train.base_price - train.checkout_price
test['discount'] = test.base_price - test.checkout_price

<h2>Define functions to generate year, quarter and month from week number</h2>

In [46]:
def get_year(week_num):
    return math.ceil(week_num/52)

In [120]:
def get_month(week_num):
    wk_excluding_year = week_num
    #there are 52 weeks in a year, so lets separate out the year by dividing with modulo
    if week_num > 52:
        wk_excluding_year = week_num%52
    #some months may have more than 4 weeks, to keep the math simple we will cap the week_number to 48 for any given year.
    if wk_excluding_year > 48:
        wk_excluding_year = 48
    
    if wk_excluding_year <= 4: return 1
    if wk_excluding_year > 4 and wk_excluding_year <= 8: return 2
    if wk_excluding_year > 8 and wk_excluding_year <= 12: return 3
    if wk_excluding_year > 12 and wk_excluding_year <= 16: return 4
    if wk_excluding_year > 16 and wk_excluding_year <= 20: return 5
    if wk_excluding_year > 20 and wk_excluding_year <= 24: return 6
    
    if wk_excluding_year > 24 and wk_excluding_year <= 28: return 7
    if wk_excluding_year > 28 and wk_excluding_year <= 32: return 8
    if wk_excluding_year > 32 and wk_excluding_year <= 36: return 9
    if wk_excluding_year > 36 and wk_excluding_year <= 40: return 10
    if wk_excluding_year > 40 and wk_excluding_year <= 44: return 11
    if wk_excluding_year > 44 and wk_excluding_year <= 48: return 12

In [116]:
def get_quarter(week_num):
    wk_excluding_year = week_num
    #there are 52 weeks in a year, so lets separate out the year by dividing with modulo
    if week_num > 52:
        wk_excluding_year = week_num%52
    #some months may have more than 4 weeks, to keep the math simple we will cap the week_number to 48 for any given year.
    if wk_excluding_year > 48:
        wk_excluding_year = 48
    if wk_excluding_year <= 12: return 1
    if wk_excluding_year > 12 and wk_excluding_year <= 24: return 2
    if wk_excluding_year > 24 and wk_excluding_year <= 36: return 3
    if wk_excluding_year > 36 and wk_excluding_year <= 48: return 4

In [143]:
train['year'] = train['week'].apply(get_year)
train['quarter'] = train['week'].apply(get_quarter)
train['month'] = train['week'].apply(get_month)

test['year'] = test['week'].apply(get_year)
test['quarter'] = test['week'].apply(get_quarter)
test['month'] = test['week'].apply(get_month)

In [144]:
#separate the target column and merge train dataset with test dataset
target = train[['id', 'num_orders']]
train.drop(['num_orders'], axis=1, inplace=True)
train['set']='train'
test['set']='test'
data = train.append(test, ignore_index=True)

<h1>Feature Engineering using Featuretools</h1>
<h2>Create an entity dataset</h2><br>
An EntitySet is a structure that contains multiple dataframes and relationships between them. So, let’s create an EntitySet and add the dataframe combination to it.

In [145]:
# create an entity set 'es'
es = ft.EntitySet(id = 'fooddemand')

# adding a dataframe 
es.entity_from_dataframe(entity_id = 'demand', dataframe = data, index = 'id')

Entityset: fooddemand
  Entities:
    demand [Rows: 489120, Columns: 19]
  Relationships:
    No relationships

In [146]:
es.normalize_entity(base_entity_id='demand', new_entity_id='fullfilment_center', index = 'center_id', 
additional_variables = ['city_code', 'region_code', 'center_type', 'op_area'])

es.normalize_entity(base_entity_id='demand', new_entity_id='meal_info', index = 'meal_id', 
additional_variables = ['category', 'cuisine'])

Entityset: fooddemand
  Entities:
    demand [Rows: 489120, Columns: 13]
    fullfilment_center [Rows: 77, Columns: 5]
    meal_info [Rows: 51, Columns: 3]
  Relationships:
    demand.center_id -> fullfilment_center.center_id
    demand.meal_id -> meal_info.meal_id

In [147]:
print(es)

Entityset: fooddemand
  Entities:
    demand [Rows: 489120, Columns: 13]
    fullfilment_center [Rows: 77, Columns: 5]
    meal_info [Rows: 51, Columns: 3]
  Relationships:
    demand.center_id -> fullfilment_center.center_id
    demand.meal_id -> meal_info.meal_id


In [148]:
# Specify the aggregation primitives
feature_matrix, feature_names = ft.dfs(entityset = es, target_entity = 'demand',  
                                                 agg_primitives = ['sum', 'count', 'min', 'max', 'mean', 'mode'], 
                                                 max_depth = 2, features_only = False, verbose = True)

Built 96 features
Elapsed: 00:45 | Remaining: 00:00 | Progress: 100%|██████████████████████████████████████████| Calculated: 10/10 chunks


In [149]:
#apply one hot encoding
def one_hot_encode(df, cat_attribs):
    dummies = pd.get_dummies(df[cat_attribs])
    df = df.join(dummies)
    df = df.drop(cat_attribs, axis=1)
    return df

In [150]:
#OHE - 'fullfilment_center.center_type', 'meal_info.category',	'meal_info.cuisine'
print('Feature matrix shape before one hot encoding is {}'.format(feature_matrix.shape))
ohe_cols = ['fullfilment_center.center_type', 'meal_info.category', 'meal_info.cuisine']

feature_matrix = one_hot_encode(feature_matrix, ohe_cols)
print('Feature matrix shape after one hot encoding is {}'.format(feature_matrix.shape))

Feature matrix shape before one hot encoding is (489120, 96)
Feature matrix shape after one hot encoding is (489120, 114)


In [151]:
#drop cols with nan correlations to the target variable
cols_to_drop = ['fullfilment_center.MIN(demand.week)', 'fullfilment_center.MIN(demand.emailer_for_promotion)', \
                'fullfilment_center.MIN(demand.homepage_featured)', 'fullfilment_center.MAX(demand.week)', \
                'fullfilment_center.MAX(demand.emailer_for_promotion)', 'fullfilment_center.MAX(demand.homepage_featured)', \
                'meal_info.MIN(demand.emailer_for_promotion)', 'meal_info.MIN(demand.homepage_featured)', \
                'meal_info.MAX(demand.week)', 'meal_info.MAX(demand.homepage_featured)']
feature_matrix.drop(cols_to_drop, axis=1, inplace=True)
print('Feature matrix shape after dropping cols with nan correlations to the target variable is {}'.format(feature_matrix.shape))

Feature matrix shape after dropping cols with nan correlations to the target variable is (489120, 104)


In [152]:
# cols = feature_matrix.columns

In [153]:
# with open('../data/columns.txt', 'w') as f:
#     for item in cols:
#         f.write("%s\n" % item)

In [154]:
# feature_matrix.iloc[:5,50:70]

In [155]:
#separate train and test dataset
train = feature_matrix[feature_matrix['set'] == 'train'].copy()
test = feature_matrix[feature_matrix['set'] == 'test'].copy()
#add num_orders to train using id
train = train.merge(target, on='id', how = 'left')
train.reset_index(inplace=True)
train.drop('index', axis=1, inplace=True)
test.reset_index(inplace=True)
print('Train shape is {}'.format(train.shape))
print('Test shape is {}'.format(test.shape))

Train shape is (456547, 106)
Test shape is (32573, 105)


In [156]:
#Drop - 'id', 'set', 'meal_info.MODE(demand.set)'
cols_drop = ['id', 'set', 'meal_info.MODE(demand.set)', 'fullfilment_center.MODE(demand.set)']
train.drop(cols_drop, axis=1, inplace=True)
test.drop(cols_drop, axis=1, inplace=True)
print('Train shape after dropping unwanted columns is {}'.format(train.shape))
print('Test shape after dropping unwanted columns is {}'.format(test.shape))

Train shape after dropping unwanted columns is (456547, 102)
Test shape after dropping unwanted columns is (32573, 101)


In [157]:
# Function to calculate correlations with the target for a dataframe
def target_corrs(df):

    # List of correlations
    corrs = []
    
    #create a new df with numeric columns
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    newdf = df.select_dtypes(include=numerics)
    
    # Iterate through the columns 
    for col in newdf.columns:
#         print(col)
        # Skip the target column
        if col != 'num_orders':
            # Calculate correlation with the target
            corr = newdf['num_orders'].corr(newdf[col])

            # Append the list as a tuple
            corrs.append((col, corr))
            
    # Sort by absolute magnitude of correlations
    corrs = sorted(corrs, key = lambda x: abs(x[1]), reverse = True)
    
    #free up memory
    gc.enable()
    del df, newdf
    gc.collect()
    
    return corrs

In [158]:
correlations=target_corrs(train)

In [159]:
correlations

[('homepage_featured', 0.29530754224247674),
 ('checkout_price', -0.2831033139067199),
 ('emailer_for_promotion', 0.2778266151993247),
 ('base_price', -0.2231735189298536),
 ('fullfilment_center.SUM(demand.week)', 0.18685518821322022),
 ('fullfilment_center.SUM(demand.year)', 0.18560314851388146),
 ('fullfilment_center.SUM(demand.month)', 0.18281221673236697),
 ('fullfilment_center.SUM(demand.quarter)', 0.1826473423694182),
 ('fullfilment_center.SUM(demand.checkout_price)', 0.18151682668359637),
 ('fullfilment_center.COUNT(demand)', 0.18140828972220485),
 ('fullfilment_center.SUM(demand.base_price)', 0.18076575893411698),
 ('fullfilment_center.op_area', 0.17757090332045494),
 ('discount', 0.1532132363855849),
 ('fullfilment_center.SUM(demand.homepage_featured)', 0.13582600883912246),
 ('fullfilment_center.SUM(demand.discount)', 0.13520848206030647),
 ('fullfilment_center.SUM(demand.emailer_for_promotion)',
  -0.06388738800039002),
 ('center_id', -0.053136539447447506),
 ('fullfilment_c

In [160]:
train.to_csv('../data/3.fe_with_week_train2.csv', index=False)
test.to_csv('../data/3.fe_with_week_test2.csv', index=False)